In [ ]:
import py_eddy_tracker_sample
from py_eddy_tracker.generic import flatten_line_matrix
from py_eddy_tracker.observations.tracking import TrackEddiesObservations
import holoviews as hv
import panel as pn
import numpy as np

hv.extension('bokeh', logo=False)
pn.extension()


In [ ]:
import pylook
from os.path import dirname
COAST = pylook.coast.CoastFile(f"{dirname(pylook.__file__)}/gshhs_backup/binned_GSHHS_l.nc")

In [ ]:
class MyTrackEddiesObservations(TrackEddiesObservations):
    def visu(self, intern=True):
        xname, yname = self.intern(intern)
        x, y = self[xname], self[yname]
        
        full_time = self.time
        available_time = list(np.unique(full_time))

        slider = pn.widgets.DiscretePlayer(name="date", options=available_time, value=available_time[0])

        def coast(x_range, y_range):
            if x_range is None:
                x_range = 0,0
                y_range = 0,0
            return hv.Path([COAST.lines(x_range[0],y_range[0],x_range[1],y_range[1]).get_paths()[0].vertices]).options(aspect='equal')

        def select(t, lon, lat):
            mask = full_time == t
            x_ = flatten_line_matrix(lon[mask])
            y_ = flatten_line_matrix(lat[mask])
            return np.array([x_, y_]).T

        @pn.depends(slider)
        def update(*args, **kwargs):
            date = kwargs["value"]
            return hv.Path([select(date, x, y)])

        @pn.depends(slider)
        def plot_date(value):
            return pn.pane.HTML(f"<h1>date = {np.datetime64('1950-01-01')+np.timedelta64(value)}</h1>")
        
        eddies = hv.DynamicMap(update)
        coast_ = hv.DynamicMap(coast, streams=[hv.streams.RangeXY(source=eddies)])
        return pn.Column(pn.Row(slider, plot_date), (eddies * coast_).opts(width=900, height=600))

c = MyTrackEddiesObservations.load_file(
    py_eddy_tracker_sample.get_demo_path("eddies_med_adt_allsat_dt2018/Cyclonic.zarr")
)
c.contour_lon_s[:] = (c.contour_lon_s + 180) % 360 -180

c.visu()